In [1]:
import game
import time
import copy
import pygame
import random
import constants
import components
import numpy as np

pygame 2.0.1 (SDL 2.0.14, Python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [15]:
g = game.Game()
g.update_ui()

In [3]:
%%time
while not g.is_done():
    g.game_state.random_move()
    g.update_ui()
# time.sleep(5)
# pygame.quit()

CPU times: user 1.19 s, sys: 108 ms, total: 1.29 s
Wall time: 5.19 s


In [22]:
g.game_state.restart()
g.update_ui()

restart


In [5]:
def random_ai_move(game_: game.Game, searches_per_move=10, search_length=10, update_ui=False):
    scores = np.zeros(4)
    game_state_init_copy = game_.game_state.copy()
    for first_move_i in range(4):
        game_.set_game_state(game_state_init_copy)
        first_move = constants.ARROW_KEYS[first_move_i]
        score, changed = game_.game_state.move(first_move)
        game_state_tmp1_copy = game_.game_state.copy()
        if update_ui:
            game_.update_ui()
        # time.sleep(1)
        if changed:
            scores[first_move_i] += score
        else:
            continue
        max_cumulative_score = 0
        for later_moves in range(searches_per_move):
            move_number = 1
            game_.set_game_state(game_state_tmp1_copy)
            changed = True
            score_cumulative = 0
            while changed and move_number < search_length:
                score, changed = game_.game_state.random_move()
                if update_ui:
                    game_.update_ui()
                if changed:
                    score_cumulative += score
                    scores[first_move_i] += score
                    move_number += 1
            max_cumulative_score = max(max_cumulative_score, score_cumulative)
        # scores[first_move_i] += max_cumulative_score
    # restore original game state
    game_.set_game_state(game_state_init_copy)
        
    best_move_index = np.argmax(scores)
    best_move = constants.ARROW_KEYS[best_move_index]
    return best_move
    

In [6]:
%%time
seed_performance_dict = {}
for seed in range(100):
    print(f"seed {seed}")
    constants.RANDOM_SEED = seed
    g.game_state.restart()
    g.update_ui()
    for i in range(1500):
        move = random_ai_move(g, searches_per_move=20, search_length=10)
        g.game_state.move(move)
        g.update_ui()
        if g.game_state.is_done:
            max_val = np.max(g.game_state.get_board().get_value_board())
            print(f"done, {i} iters, max_val: {max_val}")
            g.update_ui()
            seed_performance_dict[seed] = max_val
            if 2048 in g.game_state.get_board().get_value_board():
                print("success")
            break


seed 0
restart
done, 480 iters, max_val: 512
seed 1
restart
done, 763 iters, max_val: 1024
seed 2
restart
done, 724 iters, max_val: 1024
seed 3
restart
done, 392 iters, max_val: 512
seed 4
restart
done, 721 iters, max_val: 1024
seed 5
restart
done, 950 iters, max_val: 1024
seed 6
restart
done, 479 iters, max_val: 512
seed 7
restart
done, 737 iters, max_val: 1024
seed 8
restart
done, 935 iters, max_val: 1024
seed 9
restart
done, 278 iters, max_val: 256
seed 10
restart
done, 920 iters, max_val: 1024
seed 11
restart
done, 803 iters, max_val: 1024
seed 12
restart
done, 507 iters, max_val: 512
seed 13
restart
done, 243 iters, max_val: 256
seed 14
restart
done, 474 iters, max_val: 512
seed 15
restart
done, 382 iters, max_val: 512
seed 16
restart
done, 862 iters, max_val: 1024
seed 17
restart
done, 502 iters, max_val: 512
seed 18
restart
done, 515 iters, max_val: 512
seed 19
restart
done, 718 iters, max_val: 1024
seed 20
restart
done, 380 iters, max_val: 512
seed 21
restart
done, 729 iters, m

In [26]:
%%time
constants.RANDOM_SEED = 86
g.game_state.restart()
g.update_ui()
for i in range(1500):
    move = random_ai_move(g, searches_per_move=20, search_length=10)
    g.game_state.move(move)
    g.update_ui()
    if g.game_state.is_done:
        max_val = np.max(g.game_state.get_board().get_value_board())
        print(f"done, {i} iters, max_val: {max_val}")
        g.update_ui()
        seed_performance_dict[seed] = max_val
        break
    if 2048 in g.game_state.get_board().get_value_board():
        print(f"success in {i} moves")
        break

restart
success in 962 moves
CPU times: user 33 s, sys: 225 ms, total: 33.2 s
Wall time: 35.5 s


In [17]:
2048 in g.game_state.get_board().get_value_board()

True

In [7]:
np.max(g.game_state.get_board().get_value_board())

1024

In [10]:
max([seed_performance_dict[key] for key in seed_performance_dict.keys()])

2048

In [11]:
seed_performance_dict

{0: 512,
 1: 1024,
 2: 1024,
 3: 512,
 4: 1024,
 5: 1024,
 6: 512,
 7: 1024,
 8: 1024,
 9: 256,
 10: 1024,
 11: 1024,
 12: 512,
 13: 256,
 14: 512,
 15: 512,
 16: 1024,
 17: 512,
 18: 512,
 19: 1024,
 20: 512,
 21: 1024,
 22: 512,
 23: 1024,
 24: 512,
 25: 512,
 26: 512,
 27: 1024,
 28: 512,
 29: 512,
 30: 512,
 31: 512,
 32: 512,
 33: 512,
 34: 512,
 35: 1024,
 36: 1024,
 37: 1024,
 38: 512,
 39: 512,
 40: 1024,
 41: 1024,
 42: 512,
 43: 512,
 44: 512,
 45: 512,
 46: 512,
 47: 256,
 48: 512,
 49: 1024,
 50: 1024,
 51: 512,
 52: 1024,
 53: 1024,
 54: 512,
 55: 256,
 56: 1024,
 57: 512,
 58: 1024,
 59: 512,
 60: 512,
 61: 1024,
 62: 1024,
 63: 1024,
 64: 512,
 65: 512,
 66: 1024,
 67: 256,
 68: 1024,
 69: 1024,
 70: 1024,
 71: 1024,
 72: 512,
 73: 512,
 74: 1024,
 75: 512,
 76: 512,
 77: 512,
 78: 512,
 79: 1024,
 80: 512,
 81: 512,
 82: 1024,
 83: 512,
 84: 1024,
 85: 1024,
 86: 2048,
 87: 1024,
 88: 512,
 89: 512,
 90: 1024,
 91: 1024,
 92: 1024,
 93: 512,
 94: 512,
 95: 1024,
 96: 25

In [28]:
from typing import List

In [33]:
class Solution:
    def possible(self, board, row, col, value) -> bool:
        # verify row
        for c in range(9):
            if board[row][c] == value:
                return False
        # verify col
        for r in range(9):
            if board[r][col] == value:
                return False
        # verify square
        nr = row // 3 * 3
        nc = col // 3 * 3
        for r in range(nr, nr + 3):
            for c in range(nc, nc + 3):
                if board[r][c] == value:
                    return False
        return True
        
    def solveSudoku(self, board: List[List[str]]) -> None:
        """
        Do not return anything, modify board in-place instead.
        """
        for row in range(9):
            for col in range(9):
                if board[row][col] == '.':
                    # empty
                    for val in range(1, 10):
                        if self.possible(board, row, col, str(val)):
                            board[row][col] = str(val)
                            if self.solveSudoku(board):
                                return 1
                            else:
                                board[row][col] = '.'
                    return 0
        print(board)
        return 1

In [41]:
board = [["5","3",".",".","7",".",".",".","."],["6",".",".","1","9","5",".",".","."],[".","9","8",".",".",".",".","6","."],["8",".",".",".","6",".",".",".","3"],["4",".",".","8",".","3",".",".","1"],["7",".",".",".","2",".",".",".","6"],[".","6",".",".",".",".","2","8","."],[".",".",".","4","1","9",".",".","5"],[".",".",".",".","8",".",".","7","9"]]
# board = [[".",".",".",".",".",".",".",".","."],[".",".",".",".",".",".",".",".","."],[".",".",".",".",".",".",".",".","."],[".",".",".",".",".",".",".",".","."],[".",".",".",".",".",".",".",".","."],[".",".",".",".",".",".",".",".","."],[".",".",".",".",".",".",".",".","."],[".",".",".",".",".",".",".",".","."],[".",".",".",".",".",".",".",".","."]]

In [42]:
sol = Solution()

In [43]:
%%time
sol.solveSudoku(board)

[['5', '3', '4', '6', '7', '8', '9', '1', '2'], ['6', '7', '2', '1', '9', '5', '3', '4', '8'], ['1', '9', '8', '3', '4', '2', '5', '6', '7'], ['8', '5', '9', '7', '6', '1', '4', '2', '3'], ['4', '2', '6', '8', '5', '3', '7', '9', '1'], ['7', '1', '3', '9', '2', '4', '8', '5', '6'], ['9', '6', '1', '5', '3', '7', '2', '8', '4'], ['2', '8', '7', '4', '1', '9', '6', '3', '5'], ['3', '4', '5', '2', '8', '6', '1', '7', '9']]
CPU times: user 47.7 ms, sys: 0 ns, total: 47.7 ms
Wall time: 46.8 ms


1